### In this notebook we perform federated learning

In federated learning each base station has access only to it's private dataset, however they collaborate together to train a model that has satifactory results on data from any other base station.

In [117]:
import sys
import os

from pathlib import Path
import pandas as pd

parent = Path(os.path.abspath("")).resolve().parents[0]
if parent not in sys.path:
    sys.path.insert(0, str(parent))

In [118]:
import copy

import random

from collections import OrderedDict

import numpy as np
import torch

from argparse import Namespace

In [119]:
from ml.utils.data_utils import read_data, generate_time_lags, time_to_feature, handle_nans, to_Xy, \
    to_torch_dataset, to_timeseries_rep, assign_statistics, \
    to_train_val, scale_features, get_data_by_area, remove_identifiers, get_exogenous_data_by_area, handle_outliers

In [120]:
from ml.utils.train_utils import train, test

In [121]:
from ml.models.mlp import MLP
from ml.models.rnn import RNN
from ml.models.lstm import LSTM
from ml.models.gru import GRU
from ml.models.cnn import CNN
from ml.models.rnn_autoencoder import DualAttentionAutoEncoder

In [122]:
from ml.fl.defaults import create_regression_client
from ml.fl.client_proxy import SimpleClientProxy
from ml.fl.server.server import Server

In [123]:
args = Namespace(
    data_path='../dataset/full_dataset.csv', # dataset
    data_path_test=['../dataset/upstream_test.csv'], # test dataset
    test_size=0.3, # validation size
    targets=['temp','pH','DissolvedOxygen','Conductivity','Turbidity','AmmoniaNitrogen'], # the target columns
    num_lags=10, # the number of past observations to feed as input

    identifier='District', # the column name that identifies a bs

    nan_constant=0, # the constant to transform nan values
    x_scaler='minmax', # x_scaler
    y_scaler='minmax', # y_scaler
    outlier_detection=None, # whether to perform flooring and capping

    
    criterion='mse', # optimization criterion, mse or l1
    fl_rounds=5, # the number of federated rounds
    fraction=1., # the percentage of available client to consider for random selection
    aggregation="avg", # federated aggregation algorithm
    epochs=3, # the number of maximum local epochs
    lr=0.001, # learning rate
    optimizer='adam', # the optimizer, it can be sgd or adam
    batch_size=128, # the batch size to use
    local_early_stopping=False, # whether to use early stopping
    local_patience=50, # patience value for the early stopping parameter (if specified)
    max_grad_norm=0.0, # whether to clip grad norm
    reg1=0.0, # l1 regularization
    reg2=0.0, # l2 regularization

    cuda=True, # whether to use gpu
    
    seed=0, # reproducibility

    assign_stats=None, # whether to use statistics as exogenous data, ["mean", "median", "std", "variance", "kurtosis", "skew"]
    use_time_features=False # whether to use datetime features
)

In [124]:
print(f"Script arguments: {args}\n")

Script arguments: Namespace(aggregation='avg', assign_stats=None, batch_size=128, criterion='mse', cuda=True, data_path='../dataset/full_dataset.csv', data_path_test=['../dataset/upstream_test.csv'], epochs=3, fl_rounds=5, fraction=1.0, identifier='District', local_early_stopping=False, local_patience=50, lr=0.001, max_grad_norm=0.0, nan_constant=0, num_lags=10, optimizer='adam', outlier_detection=None, reg1=0.0, reg2=0.0, seed=0, targets=['temp', 'pH', 'DissolvedOxygen', 'Conductivity', 'Turbidity', 'AmmoniaNitrogen'], test_size=0.3, use_time_features=False, x_scaler='minmax', y_scaler='minmax')



In [125]:
device = "cuda" if args.cuda and torch.cuda.is_available() else "cpu"
print(f"Using {device}")

Using cuda


In [126]:
# Outlier detection specification
if args.outlier_detection is not None:
    outlier_columns = ['Conductivity', 'Turbidity', 'pH', 'DissolvedOxygen']
    outlier_kwargs = {"upstream": (10, 90), "midstream": (10, 90), "downstream": (5, 95)}
    args.outlier_columns = outlier_columns
    args.outlier_kwargs = outlier_kwargs

In [127]:
def seed_all():
    # ensure reproducibility
    random.seed(args.seed)
    np.random.seed(args.seed)
    torch.manual_seed(args.seed)
    torch.cuda.manual_seed_all(args.seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

In [128]:
seed_all()

### The pre-processing method is almost equivalent to centralized learning. The only difference is that the scaling operations are performed individually on each base station. In contrast, in centralized learning the scaling is performed by considering the combined data from all base stations.

In [129]:
def make_preprocessing():
    """Preprocess a given .csv"""
    # read data
    df = read_data(args.data_path)
    # handle nans
    df = handle_nans(train_data=df, constant=args.nan_constant,
                     identifier=args.identifier)
    # split to train/validation
    train_data, val_data = to_train_val(df)
    
    # handle outliers (if specified)
    if args.outlier_detection is not None:
        train_data = handle_outliers(df=train_data, columns=args.outlier_columns,
                                     identifier=args.identifier, kwargs=args.outlier_kwargs)
    
    # get X and y
    X_train, X_val, y_train, y_val = to_Xy(train_data=train_data, val_data=val_data,
                                          targets=args.targets)
    
    # scale X
    X_train, X_val, x_scalers = scale_features(train_data=X_train, val_data=X_val,
                                              scaler=args.x_scaler,
                                              per_area=True, # the features are scaled locally
                                              identifier=args.identifier)
    # scale y
    y_train, y_val, y_scalers = scale_features(train_data=y_train, val_data=y_val,
                                              scaler=args.y_scaler, 
                                              per_area=True,
                                              identifier=args.identifier)
    
    # generate time lags
    X_train = generate_time_lags(X_train, args.num_lags)
    X_val = generate_time_lags(X_val, args.num_lags)
    y_train = generate_time_lags(y_train, args.num_lags, is_y=True)
    y_val = generate_time_lags(y_val, args.num_lags, is_y=True)
    
    # get datetime features as exogenous data
    date_time_df_train = time_to_feature(
        X_train, args.use_time_features, identifier=args.identifier
    )
    date_time_df_val = time_to_feature(
        X_val, args.use_time_features, identifier=args.identifier
    )
    
    # get statistics as exogenous data
    stats_df_train = assign_statistics(X_train, args.assign_stats, args.num_lags,
                                       targets=args.targets, identifier=args.identifier)
    stats_df_val = assign_statistics(X_val, args.assign_stats, args.num_lags, 
                                       targets=args.targets, identifier=args.identifier)
    
    # concat the exogenous features (if any) to a single dataframe
    if date_time_df_train is not None or stats_df_train is not None:
        exogenous_data_train = pd.concat([date_time_df_train, stats_df_train], axis=1)
        # remove duplicate columns (if any)
        exogenous_data_train = exogenous_data_train.loc[:, ~exogenous_data_train.columns.duplicated()].copy()
        assert len(exogenous_data_train) == len(X_train) == len(y_train)
    else:
        exogenous_data_train = None
    if date_time_df_val is not None or stats_df_val is not None:
        exogenous_data_val = pd.concat([date_time_df_val, stats_df_val], axis=1)
        exogenous_data_val = exogenous_data_val.loc[:, ~exogenous_data_val.columns.duplicated()].copy()
        assert len(exogenous_data_val) == len(X_val) == len(y_val)
    else:
        exogenous_data_val = None
        
    return X_train, X_val, y_train, y_val, exogenous_data_train, exogenous_data_val, x_scalers, y_scalers

In [130]:
X_train, X_val, y_train, y_val, exogenous_data_train, exogenous_data_val, x_scalers, y_scalers = make_preprocessing()

INFO logger 2024-06-03 08:14:19,873 | data_utils.py:383 | Observations info in upstream
INFO logger 2024-06-03 08:14:19,874 | data_utils.py:384 | 	Total number of samples:  4863
INFO logger 2024-06-03 08:14:19,874 | data_utils.py:385 | 	Number of samples for training: 3891
INFO logger 2024-06-03 08:14:19,875 | data_utils.py:386 | 	Number of samples for validation:  972
INFO logger 2024-06-03 08:14:19,877 | data_utils.py:383 | Observations info in midstream
INFO logger 2024-06-03 08:14:19,878 | data_utils.py:384 | 	Total number of samples:  4930
INFO logger 2024-06-03 08:14:19,878 | data_utils.py:385 | 	Number of samples for training: 3944
INFO logger 2024-06-03 08:14:19,878 | data_utils.py:386 | 	Number of samples for validation:  986
INFO logger 2024-06-03 08:14:19,880 | data_utils.py:383 | Observations info in downstream
INFO logger 2024-06-03 08:14:19,880 | data_utils.py:384 | 	Total number of samples:  4920
INFO logger 2024-06-03 08:14:19,880 | data_utils.py:385 | 	Number of sample

In [131]:
X_train.head()

,TotalNitrogen_lag-10,TotalPhosphorus_lag-10,AmmoniaNitrogen_lag-10,PermanganateIndex_lag-10,Turbidity_lag-10,Conductivity_lag-10,DissolvedOxygen_lag-10,pH_lag-10,temp_lag-10,TotalNitrogen_lag-9,...,TotalNitrogen_lag-1,TotalPhosphorus_lag-1,AmmoniaNitrogen_lag-1,PermanganateIndex_lag-1,Turbidity_lag-1,Conductivity_lag-1,DissolvedOxygen_lag-1,pH_lag-1,temp_lag-1,District
time,,,,,,,,,,,,,,,,,,,,,
2020-11-10 16:00:00,0.053548,0.487918,0.454747,0.202132,0.004535,0.185411,0.314404,0.227700,0.671883,0.053548,...,0.054376,0.490246,0.469831,0.184912,0.007406,0.187004,0.316469,0.225352,0.661090,upstream
2020-11-10 20:00:00,0.053548,0.487918,0.454747,0.202132,0.005683,0.185266,0.278265,0.194836,0.663788,0.052996,...,0.052996,0.490246,0.463620,0.191472,0.005454,0.189756,0.352091,0.262911,0.679978,upstream
2020-11-11 00:00:00,0.052996,0.489781,0.468944,0.179582,0.005683,0.185266,0.277233,0.194836,0.658392,0.054376,...,0.057137,0.492574,0.459184,0.181222,0.004765,0.188742,0.325245,0.230047,0.666487,upstream
2020-11-11 04:00:00,0.054376,0.489781,0.468944,0.188602,0.004995,0.186714,0.323180,0.248826,0.677280,0.055756,...,0.052720,0.488850,0.458740,0.187782,0.005396,0.187438,0.304595,0.206573,0.658392,upstream
2020-11-11 08:00:00,0.055756,0.488850,0.458740,0.192292,0.004995,0.186714,0.347444,0.248826,0.685375,0.055480,...,0.052720,0.488850,0.458740,0.187782,0.005798,0.187004,0.293237,0.199530,0.655693,upstream


In [132]:
y_train.head()

,temp,pH,DissolvedOxygen,Conductivity,Turbidity,AmmoniaNitrogen,District
time,,,,,,,
2020-11-10 16:00:00,0.679978,0.262911,0.352091,0.189756,0.005454,0.463620,upstream
2020-11-10 20:00:00,0.666487,0.230047,0.325245,0.188742,0.004765,0.459184,upstream
2020-11-11 00:00:00,0.658392,0.206573,0.304595,0.187438,0.005396,0.458740,upstream
2020-11-11 04:00:00,0.655693,0.199530,0.293237,0.187004,0.005798,0.458740,upstream
2020-11-11 08:00:00,0.652995,0.197183,0.290139,0.187438,0.006946,0.462733,upstream


In [133]:
x_scalers, y_scalers

({'upstream': MinMaxScaler(),
  'midstream': MinMaxScaler(),
  'downstream': MinMaxScaler()},
 {'upstream': MinMaxScaler(),
  'midstream': MinMaxScaler(),
  'downstream': MinMaxScaler()})

### Postprocessing in a same manner with centalized learning

In [134]:
def make_postprocessing(X_train, X_val, y_train, y_val, exogenous_data_train, exogenous_data_val, x_scalers, y_scalers):
    """Make data ready to be fed into ml algorithms"""
    # if there are more than one specified areas, get the data per area
    if X_train[args.identifier].nunique() != 1:
        area_X_train, area_X_val, area_y_train, area_y_val = get_data_by_area(X_train, X_val,
                                                                              y_train, y_val, 
                                                                              identifier=args.identifier)
    else:
        area_X_train, area_X_val, area_y_train, area_y_val = None, None, None, None

    # Get the exogenous data per area.
    if exogenous_data_train is not None:
        exogenous_data_train, exogenous_data_val = get_exogenous_data_by_area(exogenous_data_train,
                                                                              exogenous_data_val)
    # transform to np
    if area_X_train is not None:
        for area in area_X_train:
            tmp_X_train, tmp_y_train, tmp_X_val, tmp_y_val = remove_identifiers(
                area_X_train[area], area_y_train[area], area_X_val[area], area_y_val[area])
            tmp_X_train, tmp_y_train = tmp_X_train.to_numpy(), tmp_y_train.to_numpy()
            tmp_X_val, tmp_y_val = tmp_X_val.to_numpy(), tmp_y_val.to_numpy()
            area_X_train[area] = tmp_X_train
            area_X_val[area] = tmp_X_val
            area_y_train[area] = tmp_y_train
            area_y_val[area] = tmp_y_val
    
    if exogenous_data_train is not None:
        for area in exogenous_data_train:
            exogenous_data_train[area] = exogenous_data_train[area].to_numpy()
            exogenous_data_val[area] = exogenous_data_val[area].to_numpy()
    
    # remove identifiers from features, targets
    X_train, y_train, X_val, y_val = remove_identifiers(X_train, y_train, X_val, y_val)
    assert len(X_train.columns) == len(X_val.columns)
    
    num_features = len(X_train.columns) // args.num_lags
    
    # to timeseries representation
    X_train = to_timeseries_rep(X_train.to_numpy(), num_lags=args.num_lags,
                                            num_features=num_features)
    X_val = to_timeseries_rep(X_val.to_numpy(), num_lags=args.num_lags,
                                          num_features=num_features)
    
    if area_X_train is not None:
        area_X_train = to_timeseries_rep(area_X_train, num_lags=args.num_lags,
                                                     num_features=num_features)
        area_X_val = to_timeseries_rep(area_X_val, num_lags=args.num_lags,
                                                   num_features=num_features)
    
    # transform targets to numpy
    y_train, y_val = y_train.to_numpy(), y_val.to_numpy()
    
    if exogenous_data_train is not None:
        exogenous_data_train_combined, exogenous_data_val_combined = [], []
        for area in exogenous_data_train:
            exogenous_data_train_combined.extend(exogenous_data_train[area])
            exogenous_data_val_combined.extend(exogenous_data_val[area])
        exogenous_data_train_combined = np.stack(exogenous_data_train_combined)
        exogenous_data_val_combined = np.stack(exogenous_data_val_combined)
        exogenous_data_train["all"] = exogenous_data_train_combined
        exogenous_data_val["all"] = exogenous_data_val_combined
    return X_train, X_val, y_train, y_val, area_X_train, area_X_val, area_y_train, area_y_val, exogenous_data_train, exogenous_data_val

In [135]:
X_train, X_val, y_train, y_val, client_X_train, client_X_val, client_y_train, client_y_val, exogenous_data_train, exogenous_data_val = make_postprocessing(X_train, X_val, y_train, y_val, exogenous_data_train, exogenous_data_val, x_scalers, y_scalers)

In [136]:
client_X_train.keys()

dict_keys(['upstream', 'midstream', 'downstream'])

In [137]:
client_X_val.keys()

dict_keys(['upstream', 'midstream', 'downstream'])

In [138]:
for client in client_X_train:
    print(f"\nClient: {client}")
    print(f"X_train shape: {client_X_train[client].shape}, y_train shape: {client_y_train[client].shape}")
    print(f"X_val shape: {client_X_val[client].shape}, y_val shape: {client_y_val[client].shape}")


Client: upstream
X_train shape: (3881, 10, 9, 1), y_train shape: (3881, 6)
X_val shape: (962, 10, 9, 1), y_val shape: (962, 6)

Client: midstream
X_train shape: (3934, 10, 9, 1), y_train shape: (3934, 6)
X_val shape: (976, 10, 9, 1), y_val shape: (976, 6)

Client: downstream
X_train shape: (3926, 10, 9, 1), y_train shape: (3926, 6)
X_val shape: (974, 10, 9, 1), y_val shape: (974, 6)


In [139]:
def get_input_dims(X_train, exogenous_data_train):
    if args.model_name == "mlp":
        input_dim = X_train.shape[1] * X_train.shape[2]
    else:
        input_dim = X_train.shape[2]
    
    if exogenous_data_train is not None:
        if len(exogenous_data_train) == 1:
            cid = next(iter(exogenous_data_train.keys()))
            exogenous_dim = exogenous_data_train[cid].shape[1]
        else:
            exogenous_dim = exogenous_data_train["all"].shape[1]
    else:
        exogenous_dim = 0
    
    return input_dim, exogenous_dim

In [140]:
def get_model(model: str,
              input_dim: int,
              out_dim: int,
              lags: int = 10,
              exogenous_dim: int = 0,
              seed=0):
    if model == "mlp":
        model = MLP(input_dim=input_dim, layer_units=[256, 128, 64], num_outputs=out_dim)
    elif model == "rnn":
        model = RNN(input_dim=input_dim, rnn_hidden_size=128, num_rnn_layers=1, rnn_dropout=0.0,
                    layer_units=[128], num_outputs=out_dim, matrix_rep=True, exogenous_dim=exogenous_dim)
    elif model == "lstm":
        model = LSTM(input_dim=input_dim, lstm_hidden_size=128, num_lstm_layers=1, lstm_dropout=0.0,
                     layer_units=[128], num_outputs=out_dim, matrix_rep=True, exogenous_dim=exogenous_dim)
    elif model == "gru":
        model = GRU(input_dim=input_dim, gru_hidden_size=128, num_gru_layers=1, gru_dropout=0.0,
                    layer_units=[128], num_outputs=out_dim, matrix_rep=True, exogenous_dim=exogenous_dim)
    elif model == "cnn":
        model = CNN(num_features=input_dim, lags=lags, exogenous_dim=exogenous_dim, out_dim=out_dim)
    elif model == "da_encoder_decoder":
        model = DualAttentionAutoEncoder(input_dim=input_dim, architecture="lstm", matrix_rep=True)
    else:
        raise NotImplementedError("Specified model is not implemented. Plese define your own model or choose one from ['mlp', 'rnn', 'lstm', 'gru', 'cnn', 'da_encoder_decoder']")
    return model

In [141]:
# define the model
args.model_name = "cnn"

input_dim, exogenous_dim = get_input_dims(X_train, exogenous_data_train)

print(input_dim, exogenous_dim)

model = get_model(model=args.model_name,
                  input_dim=input_dim,
                  out_dim=y_train.shape[1],
                  lags=args.num_lags,
                  exogenous_dim=exogenous_dim,
                  seed=args.seed)

9 0


In [142]:
model

CNN(
  (activation): ReLU()
  (conv1): Conv2d(1, 16, kernel_size=(16, 3), stride=(1, 1), padding=same)
  (conv2): Conv2d(16, 16, kernel_size=(3, 5), stride=(1, 1), padding=same)
  (conv3): Conv2d(16, 32, kernel_size=(4, 3), stride=(1, 1), padding=same)
  (conv4): Conv2d(32, 32, kernel_size=(4, 3), stride=(1, 1), padding=same)
  (pool): AvgPool2d(kernel_size=(2, 1), stride=(2, 1), padding=0)
  (fc): Linear(in_features=1440, out_features=6, bias=True)
)

### Fit function initiates the training process of every base station local model and then performs parameters aggregation on a central server for N specified federated epochs.

In [143]:
def fit(model, X_train, y_train, X_val, y_val, 
        exogenous_data_train=None, exogenous_data_val=None, 
        idxs=[0,1,3,7,8,2], # the indices of our targets in X
        log_per=1,
        client_creation_fn = None, # client specification
        local_train_params=None, # local params
        aggregation_params=None, # aggregation params
        use_carbontracker=False
       ):
    # client creation definition
    if client_creation_fn is None:
        client_creation_fn = create_regression_client
    # local params
    if local_train_params is None:
        local_train_params = {
            "epochs": args.epochs, "optimizer": args.optimizer, "lr": args.lr,
            "criterion": args.criterion, "early_stopping": args.local_early_stopping,
            "patience": args.local_patience, "device": device
        }
    
    train_loaders, val_loaders = [], []
    
    # get data per client
    for client in X_train:
        if client == "all":
            continue
        if exogenous_data_train is not None:
            tmp_exogenous_data_train = exogenous_data_train[client]
            tmp_exogenous_data_val = tmp_exogenous_data_val[client]
        else:
            tmp_exogenous_data_train = None
            tmp_exogenous_data_val = None
    
        num_features = len(X_train[client][0][0])
        
        # to torch loader
        train_loaders.append(
            to_torch_dataset(
                X_train[client], y_train[client],
                num_lags=args.num_lags,
                num_features=num_features,
                exogenous_data=tmp_exogenous_data_train,
                indices=idxs,
                batch_size=args.batch_size,
                shuffle=False
            )
        )
        val_loaders.append(
            to_torch_dataset(
                X_val[client], y_val[client],
                num_lags=args.num_lags,
                exogenous_data=tmp_exogenous_data_val,
                indices=idxs,
                batch_size=args.batch_size,
                shuffle=False
            )
            
        )
        
    # create clients with their local data
    cids = [k for k in X_train.keys() if k != "all"]
    clients = [
        client_creation_fn(
            cid=cid, # client id
            model=model, # the global model
            train_loader=train_loader, # the local train loader
            test_loader=val_loader, # the local val loader
            local_params=local_train_params # local parameters
        )
        for cid, train_loader, val_loader in zip(cids, train_loaders, val_loaders)
    ]
    
    # represent clients to server
    client_proxies = [
        SimpleClientProxy(cid, client) for cid, client in zip(cids, clients)
    ]
    
    # represent the server
    server = Server(
        client_proxies=client_proxies, # the client representations
        aggregation=args.aggregation, # the aggregation algorithm
        aggregation_params=aggregation_params, # aggregation specific params
        local_params_fn=None, # we can change the local params on demand
    )
    # Note that the client manager instance will be initialized automatically. You can define your own client manager.

    # train with FL
    model_params, history = server.fit(args.fl_rounds, args.fraction, use_carbontracker=use_carbontracker)
    
    params_dict = zip(model.state_dict().keys(), model_params)
    state_dict = OrderedDict({k: torch.Tensor(v) for k, v in params_dict})
    model = copy.deepcopy(model)
    model.load_state_dict(state_dict, strict=True)
    
    return model, history

In [144]:
# federated local params
local_train_params = {"epochs": args.epochs, "optimizer": args.optimizer, "lr": args.lr,
                      "criterion": args.criterion, "early_stopping": args.local_early_stopping,
                      "patience": args.local_patience, "device": device
                      }

In [145]:
global_model, history = fit(
    model,
    client_X_train,
    client_y_train, 
    client_X_val, 
    client_y_val, 
    local_train_params=local_train_params
)

INFO logger 2024-06-03 08:14:20,552 | server.py:62 | Initializing client manager...
INFO logger 2024-06-03 08:14:20,552 | server.py:69 | Registering clients...
INFO logger 2024-06-03 08:14:20,553 | client_manager.py:66 | Registered client with id: upstream
INFO logger 2024-06-03 08:14:20,553 | client_manager.py:66 | Registered client with id: midstream
INFO logger 2024-06-03 08:14:20,553 | client_manager.py:66 | Registered client with id: downstream
INFO logger 2024-06-03 08:14:20,553 | server.py:73 | Client manager initialized!
INFO logger 2024-06-03 08:14:20,554 | server.py:55 | Aggregation algorithm: SimpleAvg()
INFO logger 2024-06-03 08:14:20,554 | client_manager.py:88 | Parameter c=0.0. Sampled 1 client(s): ['midstream']
INFO logger 2024-06-03 08:14:21,224 | server.py:86 | Starting FL rounds
INFO logger 2024-06-03 08:14:21,227 | client_manager.py:88 | Parameter c=1.0. Sampled 3 client(s): ['midstream', 'upstream', 'downstream']
D:\Program\anaconda\envs\fed\lib\site-packages\torch\

In [146]:
global_model.state_dict()

OrderedDict([('conv1.weight',
              tensor([[[[-4.7581e-02, -5.2498e-02,  1.5617e-02],
                        [-1.0877e-02,  1.5730e-02, -8.8933e-02],
                        [-1.0140e-01, -1.3296e-02,  4.6949e-05],
                        [-4.2465e-02, -9.1430e-02, -1.0499e-01],
                        [ 4.7291e-02,  1.6294e-02,  3.8342e-02],
                        [-2.2059e-02,  6.8426e-02, -2.3353e-02],
                        [ 4.5175e-02, -4.0454e-02, -3.5840e-02],
                        [-2.2462e-03, -5.1156e-02,  3.8547e-02],
                        [ 3.5015e-02, -4.5142e-02,  2.1954e-02],
                        [-1.4889e-02, -7.7429e-02,  2.0978e-02],
                        [-1.2930e-03,  5.9037e-02, -4.8076e-03],
                        [-3.0349e-02, -1.0640e-02,  2.5386e-02],
                        [-3.0047e-02,  4.4646e-02, -4.2805e-02],
                        [-5.4551e-02,  9.2332e-03,  1.2898e-01],
                        [-3.4249e-02, -7.5406e-02, -2.0197e-

In [147]:
history


History (client, train losses):
	midstream: {1: 0.00011136518302693183, 2: 9.300731635975865e-05, 3: 7.146666680822972e-05, 4: 7.697379786022156e-05, 5: 9.065095554700146e-05}
	upstream: {1: 0.00011085080988589973, 2: 6.157865337341163e-05, 3: 5.460808473439775e-05, 4: 4.988717607802504e-05, 5: 4.173709135776142e-05}
	downstream: {1: 0.0001315119847326985, 2: 0.00010988663469730169, 3: 7.605883262015336e-05, 4: 7.17156449611803e-05, 5: 6.438381042401674e-05}

History (client, train metrics):
	midstream: {1: {'MSE': 0.014151853, 'RMSE': 0.11896155924467672, 'MAE': 0.0936552, 'R^2': -0.09903742291344485, 'NRMSE': 0.9849771269000459}, 2: {'MSE': 0.011831862, 'RMSE': 0.10877436242357283, 'MAE': 0.07801473, 'R^2': 0.17629388375472513, 'NRMSE': 0.8081816479167916}, 3: {'MSE': 0.009109822, 'RMSE': 0.09544538805982739, 'MAE': 0.06751217, 'R^2': 0.4200972250579051, 'NRMSE': 0.7008272683008272}, 4: {'MSE': 0.009808536, 'RMSE': 0.09903805171561791, 'MAE': 0.06858187, 'R^2': 0.3307158447738638, '